# Setup

## Imports and "Globals"

In [ ]:
from virl2_client import ClientLibrary
import getpass
import jinja2
import json
import re
import textwrap
from passlib.hash import sha512_crypt
import os,time

In [ ]:
# This will be the ip address of the gateway router, to the lab.
# In most cases, should be set to 'dhcp'.  If you do not have DHCP service in the segment hosting CML
# then replace with IP address and netmask in the form of '123.123.123.10 255.255.255.0'
LAB_IP = 'dhcp'

In [ ]:
LAB_TITLE      = 'fabric'
LAB_DESCR      = ''
LAB_NOTES      = ''
LAB_TAGS       = ['spine','leaf','compute','oob']
USERS = {
    'cisco'  : dict(username='cisco' ,passwd='cisco'),
    'admin'  : dict(username='admin' ,passwd='admin'),
    'ubuntu' : dict(username='ubuntu',passwd='ubuntu')
}
DNS_NAMESERVER = 'your_dns_server'

## Password Hashing

In [ ]:
# Uses the actual password as the (not very secure) salt, making it easy to visually
# identify the password in cloud-init (never really do in production).

for user in USERS.values():
    salt = user['passwd']
    user['hash'] = sha512_crypt.using(rounds=5000,salt=salt).hash(user['passwd'])

## RSA Keys

In [ ]:
# Adjust, as needed

LAB_PRIVATE_KEY_FILE = '/Users/your_user_name/.ssh/lab_rsa'
LAB_PUBLIC_KEY_FILE  = '/Users/your_user_name/.ssh/lab_rsa.pub'

with open ('/Users/your_user_name/.ssh/lab_rsa.pub', 'r') as f:
    LAB_PUBLIC_KEY = f.read().rstrip()
LAB_PUBLIC_KEY_FOLDED = textwrap.fill(LAB_PUBLIC_KEY,80)

## Server Identification, Jump and Login Credentials

In [ ]:
VIRL_SERVER    = 'your_cml_server'

In [ ]:
# In my environment, I have to jump through intermdiary box to get to CML.
# If you do not need, set JUMP_BOX = None.

JUMP_BOX           = dict(host    = 'your_jump_box',
                          port    = '22',
                          user    = 'our_user_name',
                          id_file = '~/.ssh/id_rsa'
                         )

In [ ]:
VIRL_USER      = input('VIRL Username: ')
VIRL_PASSWD    = getpass.getpass('VIRL Password: ')

# The Build

## Session Establishment and Lab Creation

In [ ]:
client = ClientLibrary('https://{}'.format(VIRL_SERVER),
                       VIRL_USER,
                       VIRL_PASSWD,
                       ssl_verify=False
                      )
# The following began to error out, complaining that it is deprecated syntax.
#client.wait_for_lld_connected()

if client.is_system_ready(wait=True):
    None

In [ ]:
lab             = client.create_lab(LAB_TITLE)
lab.description = LAB_DESCR
lab.notes       = LAB_NOTES

## Create Nodes

In [ ]:
# Create out-of-band networking nodes.

outside = lab.create_node('outside','external_connector',50,400)
outside.config = 'bridge0'
outside.add_tag('oob')

gateway = lab.create_node(LAB_TITLE,'iosv',-50,400)
gateway.image_definition = 'iosv-159-3-m3'
gateway.add_tag('oob')

bridge1 = lab.create_node('bridge1','unmanaged_switch',-150,300)
bridge1.add_tag('oob')

bridge2 = lab.create_node('bridge2','unmanaged_switch',-150,400)
bridge2.add_tag('oob')

mgt = lab.create_node('mgt','ubuntu',-50,300)
mgt.image_definition = 'ubuntu-20-04-20210224'
mgt.add_tag('oob')


#mgt = lab.create_node('mgt','lxc',-50,300)
#mgt.add_tag('oob')

# Create a reference group.

oobs = [outside, gateway, bridge1, bridge2, mgt]

In [ ]:
# Create Spine
    
spine1 = lab.create_node('spine1','nxosv9000',-500,50)
spine1.image_definition = 'nxosv9000-9-2-4'
spine1.add_tag('spine')

spine2 = lab.create_node('spine2','nxosv9000',-400,50)
spine2.image_definition = 'nxosv9000-9-2-4'
spine2.add_tag('spine')

spine3 = lab.create_node('spine3','nxosv9000',-300,50)
spine3.image_definition = 'nxosv9000-9-2-4'
spine3.add_tag('spine')

# Create a reference group.

spines = [spine1, spine2, spine3]

In [ ]:
# Create Leaves

leaf1 = lab.create_node('leaf1','nxosv9000',-550,200)
leaf1.image_definition = 'nxosv9000-9-2-4'
leaf1.add_tag('leaf')

leaf2 = lab.create_node('leaf2','nxosv9000',-450,200)
leaf2.image_definition = 'nxosv9000-9-2-4'
leaf2.add_tag('leaf')

leaf3 = lab.create_node('leaf3','nxosv9000',-350,200)
leaf3.image_definition = 'nxosv9000-9-2-4'
leaf3.add_tag('leaf')

leaf4 = lab.create_node('leaf4','nxosv9000',-250,200)
leaf4.image_definition = 'nxosv9000-9-2-4'
leaf4.add_tag('leaf')

# Create a reference group.

leaves = [leaf1, leaf2, leaf3, leaf4]

In [ ]:
# Create a compute reference group.

compute = list()

# Create Compute Hosts

compute1 = lab.create_node('compute1','ubuntu',-550,300)
compute1.image_definition = 'ubuntu-20-04-20210224'
compute1.add_tag('compute')
compute.append(compute1)

compute2 = lab.create_node('compute2','ubuntu',-550,350)
compute2.image_definition = 'ubuntu-20-04-20210224'
compute2.add_tag('compute')
compute.append(compute2)

compute3 = lab.create_node('compute3','ubuntu',-550,400)
compute3.image_definition = 'ubuntu-20-04-20210224'
compute3.add_tag('compute')
compute.append(compute3)

compute4 = lab.create_node('compute4','ubuntu',-450,300)
compute4.image_definition = 'ubuntu-20-04-20210224'
compute4.add_tag('compute')
compute.append(compute4)

compute5 = lab.create_node('compute5','ubuntu',-450,350)
compute5.image_definition = 'ubuntu-20-04-20210224'
compute5.add_tag('compute')
compute.append(compute5)

compute6 = lab.create_node('compute6','ubuntu',-450,400)
compute6.image_definition = 'ubuntu-20-04-20210224'
compute6.add_tag('compute')
compute.append(compute6)

compute7 = lab.create_node('compute7','ubuntu',-350,300)
compute7.image_definition = 'ubuntu-20-04-20210224'
compute7.add_tag('compute')
compute.append(compute7)

compute8 = lab.create_node('compute8','ubuntu',-350,350)
compute8.image_definition = 'ubuntu-20-04-20210224'
compute8.add_tag('compute')
compute.append(compute8)

compute9 = lab.create_node('compute9','ubuntu',-350,400)
compute9.image_definition = 'ubuntu-20-04-20210224'
compute9.add_tag('compute')
compute.append(compute9)

compute10 = lab.create_node('compute10','ubuntu',-250,300)
compute10.image_definition = 'ubuntu-20-04-20210224'
compute10.add_tag('compute')
compute.append(compute10)

compute11 = lab.create_node('compute11','ubuntu',-250,350)
compute11.image_definition = 'ubuntu-20-04-20210224'
compute11.add_tag('compute')
compute.append(compute11)

compute12 = lab.create_node('compute12','ubuntu',-250,400)
compute12.image_definition = 'ubuntu-20-04-20210224'
compute12.add_tag('compute')
compute.append(compute12)

## Cable Lab

In [ ]:
# Cable OOB Together

oob_links = list()
oob_links.append ( lab.create_link(gateway.create_interface(),outside.create_interface()) )
oob_links.append ( lab.create_link(bridge1.create_interface(),gateway.create_interface()) )
oob_links.append ( lab.create_link(bridge2.create_interface(),gateway.create_interface()) )
oob_links.append ( lab.create_link(bridge1.create_interface(),mgt.create_interface()) )

In [ ]:
# Cable Spine and Leaves to OOB

for node in spines + leaves:
    oob_links.append ( lab.create_link(node.create_interface(),bridge1.create_interface()) )

In [ ]:
# Cable Compute to OOB
    
for node in compute:
    oob_links.append ( lab.create_link(node.create_interface(),bridge2.create_interface()) )

In [ ]:
# Cable Compute to Leaves
    
compute_links = list()

j=0; k=0
while j < 12:
    # create two interfaces for each node to TOR
    for i in range(0,2):
        compute_links.append ( lab.create_link(compute[j+0].create_interface(),leaves[k].create_interface()) )
        compute_links.append ( lab.create_link(compute[j+1].create_interface(),leaves[k].create_interface()) )
        compute_links.append ( lab.create_link(compute[j+2].create_interface(),leaves[k].create_interface()) )
    j+=3; k+=1

In [ ]:
# Cable Leaves to Spine
    
leaf_links = list()

for leaf in leaves:
    for spine in spines:
        leaf_links.append ( lab.create_link(leaf.create_interface(),spine.create_interface()) )

## Create Metadata Store

In [ ]:
# Create container dictionary
data = dict()

In [ ]:
# Lab specifics
data['lab'] = {
    'virl_title'             : lab.title,
    'virl_id'                : lab.id,
    'virl_base_url'          : lab.lab_base_url,
    'virl_server'            : VIRL_SERVER,
    'virl_user'              : VIRL_USER,
    'virl_description'       : lab.description,
    'virl_notes'             : lab.notes,
    'virl_node_tags'         : LAB_TAGS,
    'fabric_platform'        : 'nxos',
    'edge_configuration'     : 'l3'
}

# Common configuration, shared among nodes
data['common'] = {
    'dns_nameserver'    : DNS_NAMESERVER
}

data['users'] = USERS

data['rsa_pub_key'] = LAB_PUBLIC_KEY
data['rsa_pub_key_folded'] = LAB_PUBLIC_KEY_FOLDED

# Container for node specific configuration
data['nodes'] = dict()

In [ ]:
# Create node dictionaries for out-of-band nodes (IP Addressing as placeholder, populated later)
for node in oobs:
    if node == gateway:
        entry = dict(node_label       = node.label,
                     node_id          = node.id,
                     node_definition  = node.node_definition,
                     image_definition = node.image_definition,
                     external_address = None,
                     nat_port         = None,
                     user             = 'cisco',
                     node_tag         = node.tags()[0]
                    )
    elif node.label == 'mgt':
        entry = dict(node_label       = node.label,
                     node_id          = node.id,
                     node_definition  = node.node_definition,
                     image_definition = node.image_definition,
                     external_address = None,
                     nat_port         = '610{}'.format(node.id[1:].zfill(2)),
                     user             = 'cisco',
                     node_tag         = node.tags()[0]
                    )     
    else:
        entry = dict(node_label       = node.label,
                     node_id          = node.id,
                     node_definition  = node.node_definition,
                     image_definition = node.image_definition,
                     external_address = None,
                     nat_port         = None,
                     user             = None,
                     node_tag         = node.tags()[0]
                    )
    entry['interfaces'] = dict()
    for interface in node.interfaces():
        entry['interfaces'][interface.label] = dict(interface_id    = interface.id,
                                                    interface_label = interface.label,
                                                    ip_address      = None,
                                                    netmask         = None,
                                                    neighbor        = None
                                                   )
        if not interface.links():
            entry['interfaces'][interface.label]['associated_link_id']     = None
        else:
            entry['interfaces'][interface.label]['associated_link_id']     = interface.links()[0].id
            entry['interfaces'][interface.label]['neighbor']               = dict()
            # .peer_nodes() and .peer_interfaces return a set object
            entry['interfaces'][interface.label]['neighbor']['label']      = interface.peer_nodes().pop().label
            entry['interfaces'][interface.label]['neighbor']['interface']  = interface.peer_interfaces().pop().label
            entry['interfaces'][interface.label]['neighbor']['ip_address'] = None
    data['nodes'][node.label] = entry

In [ ]:
# Create node dictionaries for spine and leaf switches (IP Addressing as placeholder, populated later)
for node in spines + leaves:
    entry = dict(node_label       = node.label,
                 node_id          = node.id,
                 node_definition  = node.node_definition,
                 image_definition = node.image_definition,
                 external_address = None,
                 nat_port         = '610{}'.format(node.id[1:].zfill(2)),
                 user             = 'cisco',
                 node_tag         = node.tags()[0]
                )
    entry['interfaces'] = dict()
    for interface in node.interfaces():
        entry['interfaces'][interface.label] = dict(interface_id    = interface.id,
                                                    interface_label = interface.label,
                                                    ip_address      = None,
                                                    netmask         = None,
                                                    neighbor        = None
                                                   )
        if not interface.links():
            entry['interfaces'][interface.label]['associated_link_id']     = None
        else:
            entry['interfaces'][interface.label]['associated_link_id']     = interface.links()[0].id
            entry['interfaces'][interface.label]['neighbor']               = dict()
            # .peer_nodes() and .peer_interfaces return a set object
            entry['interfaces'][interface.label]['neighbor']['label']      = interface.peer_nodes().pop().label
            entry['interfaces'][interface.label]['neighbor']['interface']  = interface.peer_interfaces().pop().label
            entry['interfaces'][interface.label]['neighbor']['ip_address'] = None
    data['nodes'][node.label] = entry

In [ ]:
# Create node dictionaries for edge compute boxes (IP Addressing as placeholder, populated later)
for node in compute:
    entry = dict(node_label       = node.label,
                 node_id          = node.id,
                 node_definition  = node.node_definition,
                 image_definition = node.image_definition,
                 external_address = None,
                 nat_port         = '610{}'.format(node.id[1:].zfill(2)),
                 user             = 'cisco',
                 node_tag         = node.tags()[0]
                )
    entry['interfaces'] = dict()
    for interface in node.interfaces():
        entry['interfaces'][interface.label] = dict(interface_id    = interface.id,
                                                    interface_label = interface.label,
                                                    ip_address      = None,
                                                    netmask         = None,
                                                    neighbor        = None
                                                   )
        if not interface.links():
            entry['interfaces'][interface.label]['associated_link_id']     = None
        else:
            entry['interfaces'][interface.label]['associated_link_id']     = interface.links()[0].id
            entry['interfaces'][interface.label]['neighbor']               = dict()
            # .peer_nodes() and .peer_interfaces return a set object
            entry['interfaces'][interface.label]['neighbor']['label']      = interface.peer_nodes().pop().label
            entry['interfaces'][interface.label]['neighbor']['interface']  = interface.peer_interfaces().pop().label
            entry['interfaces'][interface.label]['neighbor']['ip_address'] = None
    data['nodes'][node.label] = entry

In [ ]:
# Assign roles to the interfaces associated with links, to aid in templating routing protocols, etc.

for link in lab.links():
    if link in leaf_links:
        data['nodes'][link.node_a.label]['interfaces'][link.interface_a.label]['role'] = 'fabric'
        data['nodes'][link.node_b.label]['interfaces'][link.interface_b.label]['role'] = 'fabric'
    elif link in compute_links:
        data['nodes'][link.node_a.label]['interfaces'][link.interface_a.label]['role'] = 'edge'
        data['nodes'][link.node_b.label]['interfaces'][link.interface_b.label]['role'] = 'edge'
    else:
        data['nodes'][link.node_a.label]['interfaces'][link.interface_a.label]['role'] = 'management'
        data['nodes'][link.node_b.label]['interfaces'][link.interface_b.label]['role'] = 'management'

In [ ]:
# Assign addressing based on role
for node in data['nodes'].values():
    for k,v in node['interfaces'].items():
        if k == 'Loopback0':
            node['interfaces'][k]['role'] = 'loopback'
            node['interfaces'][k]['ip_address'] = '172.16.0.{}'.format(node['node_id'][1:])
            node['interfaces'][k]['netmask'] = '255.255.255.255'
            
        elif v['role'] == 'fabric':
            node['interfaces'][k]['ip_address'] = '172.16.{}.{}'.format(v['associated_link_id'][1:],node['node_id'][1:])
            node['interfaces'][k]['netmask'] = '255.255.255.0'
            
        elif v['role'] == 'management' and (node['node_definition'] == 'nxosv9000' or node['node_definition'] == 'ubuntu'):
            node['interfaces'][k]['ip_address'] = '192.168.1.{}'.format(node['node_id'][1:])
            node['interfaces'][k]['netmask'] = '255.255.255.0'
        # Addressed because L3 edge.  If L2 edge, this address and netmask would be set to None.
        elif v['role'] == 'edge':
            node['interfaces'][k]['ip_address'] = '172.16.{}.{}'.format(v['associated_link_id'][1:],node['node_id'][1:])
            node['interfaces'][k]['netmask'] = '255.255.255.0'
        else:
            pass

In [ ]:
# Add an extra BVI interface to gateway node for bridging its Gig0/1 and Gig0/2 interfaces
# together, and allowing routing over the BVI.
#
# Just coded in the .1, because I alwasy set gateway node to being n1,
# where external connector is always n0
data['nodes']['fabric']['interfaces']['BVI1'] = dict(interface_id       = None,
                                                     interface_label    = None,
                                                     ip_address         = '192.168.1.1',
                                                     netmask            = '255.255.255.0',
                                                     associated_link_id = None,
                                                     neighbor           = None,
                                                     role               = 'management'
                                                    )

In [ ]:
# Loop through all nodes interfaces and fill in neighbor's IP address, if exists.
for k,v in data['nodes'].items():
    for interface in v['interfaces'].values():
        if interface['neighbor']:
            interface['neighbor']['ip_address'] = data['nodes'][interface['neighbor']['label']]['interfaces'][interface['neighbor']['interface']]['ip_address']

In [ ]:
# Add a hosts table information to metadata (including NAT port)
hosts = list()
for k,v in data['nodes'].items():
    for interface,details in v['interfaces'].items():
        if details['role'] == 'management' and details['ip_address']:
            #print (k,interface,details['ip_address'])
            hosts.append(dict(name=k,ip_address=details['ip_address'],nat_port=v['nat_port']))
data['common']['hosts'] = hosts

## Generate and Install Day0 Configurations

In [ ]:
gateway_template = '''!
!
hostname {{ node_label }}
!
{%- for host in hosts %}
ip host {{ host.name }} {{ host.ip_address }}
{%- endfor %}
!
ip domain list sandbox
ip domain name sandbox
!
no banner exec
no banner incoming
no banner login
!
enable password {{ users.cisco.passwd }}
!
aaa new-model
!
!
aaa authentication login default local
aaa authorization exec default local
aaa authorization console
!
username {{ users.cisco.username }} privilege 15 password 0 {{ users.cisco.passwd }}
!
line vty 0 4
 transport input ssh
 exec-timeout 0 0
!
!
bridge irb
!
bridge 1 protocol ieee
bridge 1 route ip
!
!
interface Loopback0
 no ip address
!
interface GigabitEthernet0/0
 description EXTERNAL CONNECTOR via {{ interfaces['GigabitEthernet0/0'].neighbor.label }}'s {{ interfaces['GigabitEthernet0/0'].neighbor.interface }}
 {%- if LAB_IP == 'dhcp' %}
 ip dhcp client client-id GigabitEthernet0/0
 ip address dhcp
 {%- else %}
 ip address {{ LAB_IP }}
 {%- endif %}
 ip nat outside
 no shutdown
!
interface GigabitEthernet0/1
 description  TO {{ interfaces['GigabitEthernet0/1'].neighbor.label }}'s {{ interfaces['GigabitEthernet0/1'].neighbor.interface }}
 no ip address
 bridge-group 1
 no shutdown
!
interface GigabitEthernet0/2
 description TO {{ interfaces['GigabitEthernet0/2'].neighbor.label }}'s {{ interfaces['GigabitEthernet0/2'].neighbor.interface }}
 no ip address
 bridge-group 1
 no shutdown
!
!
interface BVI1
 ip address {{ interfaces.BVI1.ip_address}} {{ interfaces.BVI1.netmask}}
 ip nat inside
 no shutdown
!
!
ip name-server {{ dns_nameserver }}
!
!
{%- for host in hosts %}
{%- if host.nat_port %}
ip nat inside source static tcp {{ host.ip_address }} 22 interface GigabitEthernet0/0 {{ host.nat_port }}
{%- endif %}
{%- endfor %}
ip nat inside source list NAT_ALLOWED int GigabitEthernet0/0 overload
ip access-list standard NAT_ALLOWED
 permit 0.0.0.0 255.255.255.255
!
! Routes pointing back to container rails, so they can get out to internet for apt loads, etc.
!
{%- for box in compute %}
ip route 10.{{ box.node_id[1:] }}.0.0 255.255.0.0 {{ box.interfaces['ens2'].ip_address }}
{%- endfor %}
!
ip ssh version 2
crypto key generate rsa modulus 2048
!
ip ssh pubkey-chain
  username cisco
    key-string
{{ rsa_pub_key_folded }}
    exit
  exit
exit
!
!
end
'''

In [ ]:
template = jinja2.Template(gateway_template)
gateway.config = template.render(**data['nodes']['fabric'],
                                 **data['common'],
                                 users=data['users'],
                                 compute = [node for node in data['nodes'].values() if node['node_tag'] == 'compute'],
                                 rsa_pub_key_folded=data['rsa_pub_key_folded'],
                                 LAB_IP = LAB_IP
                                )

In [ ]:
fabric_template = '''!
!
! default of 1536 needs to be reset to accomodate
! arp-supression
!
! hardware access-list tcam region racl 1536
! hardware access-list tcam region arp-ether 0
! 
hardware access-list tcam region racl 512
hardware access-list tcam region arp-ether 256 double-wide
!
hostname {{ node_label }}
!
{%- for interface in interfaces %}
interface {{ interface }}
 {%- if interfaces[interface].neighbor %}
 description TO {{ interfaces[interface].neighbor.label }}'s {{ interfaces[interface].neighbor.interface }}
 {%- endif %}
 {%- if interfaces[interface].ip_address %}
 {%- if interface != 'Loopback0' and interface != 'mgmt0' %}
 no switchport
 {%- endif %}
 ip address {{ interfaces[interface].ip_address }} {{ interfaces[interface].netmask }}

 {%- endif %}
 no shutdown
 !
{%- endfor %}
!
vrf context management
  ip domain-name sandbox
  ip domain-list sandbox
  ip name-server {{ dns_nameserver }}
  ip route 0.0.0.0/0 192.168.1.1
!
no password strength-check
username {{ users.admin.username }} password 0 {{ users.admin.passwd }} role network-admin
username {{ users.admin.username }} sshkey {{ rsa_pub_key }}
username {{ users.cisco.username }} password 0 {{ users.cisco.passwd }} role network-admin
username {{ users.cisco.username }} sshkey {{ rsa_pub_key }}
!
{%- for host in hosts %}
ip host {{ host.name }} {{ host.ip_address }}
{%- endfor %}
!
!
line console
  exec-timeout 0
line vty
  exec-timeout 0
!
end
'''

In [ ]:
for node in spines + leaves:
    template = jinja2.Template(fabric_template)
    node.config = template.render(**data['nodes'][node.label],
                                    **data['common'],
                                    users=data['users'],
                                    rsa_pub_key=data['rsa_pub_key']
                                   )

In [ ]:
compute_template = '''#cloud-config
password: {{ users.ubuntu.passwd }}
chpasswd: { expire: False }
hostname: {{ node_label }}
ssh_pwauth: True

users:
  - default
  - name: {{ users.cisco.username }}
    gecos: Cisco User
    shell: /bin/bash
    lock_passwd: False
    sudo: ALL=(ALL) NOPASSWD:ALL
    passwd: {{ users.cisco.hash }}
    ssh_authorized_keys: {{ rsa_pub_key }}

write_files:
  - content: |
      network:
        version: 2
        ethernets:
          ens2:
            optional: false
            dhcp4: false
            addresses: [ {{ interfaces['ens2'].ip_address }}/24 ]
            #To {{ interfaces['ens2'].neighbor.label }}'s' {{ interfaces['ens2'].neighbor.interface }}
            gateway4: 192.168.1.1
            nameservers:
              addresses: [ {{ dns_nameserver }} ]
              search: [ sandbox ]
          {%- if node_label != 'mgt' %}
          ens3:
            optional: false
            dhcp4: false
            addresses: [ {{ interfaces['ens3'].ip_address }}/24 ]
            #To {{ interfaces['ens3'].neighbor.label }}'s' {{ interfaces['ens3'].neighbor.interface }}
          ens4:
            optional: false
            dhcp4: false
            addresses: [ {{ interfaces['ens4'].ip_address }}/24 ]
            #To {{ interfaces['ens4'].neighbor.label }}'s' {{ interfaces['ens4'].neighbor.interface }}
          {%- endif %}
    path: /etc/netplan/60-updated-config.yaml
    permissions: '0644'

runcmd:
  - sudo netplan apply
'''

In [ ]:
for node in compute + [mgt]:
    template = jinja2.Template(compute_template)
    node.config = template.render(**data['nodes'][node.label],
                                  **data['common'],
                                  users=data['users'],
                                  rsa_pub_key=data['rsa_pub_key'],
                                  gateway = data['nodes'][gateway.label]['interfaces']['BVI1']['ip_address']
                                 )

In [ ]:
# Write topology to file

virl_topology_file = './virl-topology.yaml'
with open(virl_topology_file,'w') as f:
    f.write(lab.download())

In [ ]:
# Create a lab specific ssh-config

ssh_config_template = '''
StrictHostKeyChecking no
UserKnownHostsFile /dev/null
LogLevel QUIET
IdentityFile {{ rsa_private_key_file }}

{%- if JUMP_BOX %}

Host jump
    Hostname {{ JUMP_BOX.host }}
    User {{ JUMP_BOX.user }}
    IdentityFile {{ JUMP_BOX.id_file }}
    Port {{ JUMP_BOX.port }}
{%- endif %}

Host {{ gateway }}
    Hostname {{ gateway }}
    # Should be in DNS, if not, you might need to manipulate the above Hostname
    # to reflect the IP address.
    {%- if JUMP_BOX %}
    ProxyJump {{ JUMP_BOX.user }}@jump:{{ JUMP_BOX.port }}
    {%- endif %}
    User {{ users.cisco.username }}
    Port 22

{%- for host in ssh_host_data %}

Host {{ host.node_label }}
    Hostname {{ gateway  }}
    {%- if JUMP_BOX %}
    ProxyJump {{ JUMP_BOX.user }}@jump:{{ JUMP_BOX.port }}
    {%- endif %}
    {%- if host.user == 'admin'%}
    User {{ users.admin.username }}
    {%- elif host.user == 'ubuntu'%}
    User {{ users.ubuntu.username }}
    {%- else %}
    User {{ users.cisco.username }}
    {%- endif %}
    Port {{ host.nat_port }}

{%- endfor %}
'''

In [ ]:
ssh_host_data = [
    data['nodes'][node] for node in
        [data['nodes'][node.label]['node_label'] for node in 
             [mgt] + spines + leaves + compute]
]

In [ ]:
template = jinja2.Template(ssh_config_template)
ssh_config = template.render(ssh_host_data=ssh_host_data,
                             users=data['users'],
                             gateway=gateway.label,
                             rsa_private_key_file=LAB_PRIVATE_KEY_FILE,
                             JUMP_BOX = JUMP_BOX
                            )

In [ ]:
# Write ssh-config to file

ssh_config_file = './ssh-config'
with open(ssh_config_file,'w') as f:
    f.write(ssh_config)

In [ ]:
# Generate Ansible configuration file

ansible_config = '''
[defaults]
interpreter_python = /usr/bin/python3
inventory = ./ansible-inventory.ini
host_key_checking = False
timeout=30
forks=5

[ssh_connection]
ssh_args = -F {} -o ControlMaster=auto -o ControlPersist=60s -o PreferredAuthentications=publickey
pipelining = true
'''.format('./ssh-config')

ansible_config_file = './ansible.cfg'
with open (ansible_config_file,'w') as f:
    f.write(ansible_config)

In [ ]:
# Generate Ansible inventory file

ansible_host_data = [
    data[ 'nodes'][node] for node in
        [data['nodes'][node.label]['node_label'] for node in 
             [gateway,mgt] + spines + leaves + compute ]
]                  

ansible_inventory = dict(spines=list(),
                         leaves=list(),
                         compute=list()
                        )

for host in ansible_host_data:
    if host['node_tag'] == 'spine':
        ansible_inventory['spines'].append(host)
    elif host['node_tag'] == 'leaf':
        ansible_inventory['leaves'].append(host)
    elif host['node_tag'] == 'compute':
        ansible_inventory['compute'].append(host)
    elif host['node_label'] == gateway.label:
        ansible_inventory['gateway'] = host
    elif host['node_label'] == mgt.label:
        ansible_inventory['mgt'] = host
    else:
        print ('something went wrong, put in an exception clause later')

ansible_inventory_template = '''
[gateway]
{{ ansible_inventory.gateway.node_label }}

[gateway:vars]
ansible_user= {{ users.cisco.username }}
ansible_private_key_file={{ rsa_private_key_file }}
ansible_connection=network_cli
{%- if JUMP_BOX %}
ansible_ssh_common_args='-o ProxyCommand="ssh -W %h:%p -q {{ JUMP_BOX.user }}@{{ JUMP_BOX.host }}"'
{%- endif %}
ansible_network_os=ios

[compute]
{%- for host in ansible_inventory.compute %}
{{ host.node_label }} ansible_port={{ host.nat_port }}
{%- endfor %}

[compute:vars]
ansible_host={{ ansible_inventory.gateway.node_label }}
ansible_connection=ssh

[management]
{{ ansible_inventory.mgt.node_label }} ansible_host={{ ansible_inventory.gateway.node_label }} ansible_port={{ ansible_inventory.mgt.nat_port }} ansible_connection=ssh

[leaves]
{%- for leaf in ansible_inventory.leaves %}
{{ leaf.node_label }} ansible_port={{ leaf.nat_port }}
{%- endfor %}

[spines]
{%- for spine in ansible_inventory.spines %}
{{ spine.node_label }} ansible_port={{ spine.nat_port }}
{%- endfor %}

[nxos:children]
leaves
spines

[nxos:vars]
ansible_host={{ ansible_inventory.gateway.node_label }}
ansible_user={{ users.cisco.username }}
ansible_private_key_file={{ rsa_private_key_file }}
ansible_connection=network_cli
{%- if JUMP_BOX %}
ansible_ssh_common_args='-o ProxyCommand="ssh -W %h:%p -q {{ JUMP_BOX.user }}@{{ JUMP_BOX.host }}"'
{%- endif %}
ansible_network_os=nxos
'''

In [ ]:
template = jinja2.Template(ansible_inventory_template)
ansible_inventory_config = template.render(ansible_inventory    = ansible_inventory,
                                           users                = data['users'],
                                           rsa_private_key_file = LAB_PRIVATE_KEY_FILE,
                                           JUMP_BOX             = JUMP_BOX
                                          )

In [ ]:
ansible_inventory_file = './ansible-inventory.ini'
with open (ansible_inventory_file,'w') as f:
    f.write(ansible_inventory_config)

In [ ]:
# Start lab

lab.start()

In [ ]:
lab.state()

In [ ]:
# Capture the DHCP assigned address for gateway's interface to external world and resulting lab access.

data['nodes'][gateway.label]['interfaces']['GigabitEthernet0/0']['ip_address'] = gateway.get_interface_by_label('GigabitEthernet0/0').discovered_ipv4[0]
data['lab']['access'] = data['nodes'][gateway.label]['interfaces']['GigabitEthernet0/0']['ip_address']

In [ ]:
# Add MAC address information to interfaces in metadata

for node,node_details in data['nodes'].items():
    for interface,interface_details in node_details['interfaces'].items():
        if interface_details['interface_label']:
            data['nodes'][node]['interfaces'][interface]['mac_address'] = lab.get_node_by_label(node).get_interface_by_label(interface).discovered_mac_address
        else:
            data['nodes'][node]['interfaces'][interface]['mac_address'] = None

In [ ]:
data['lab']['virl_topology_file'] = virl_topology_file
data['lab']['ssh_config_file'] = ssh_config_file
data['lab']['ansible_config_file'] = ansible_config_file
data['lab']['ansible_inventory_file'] = ansible_inventory_file
data['lab']['working_directory'] = os.popen('pwd').read().rstrip()
data['lab']['timestamp'] = time.time()

In [ ]:
# Write metadata to JSON file

with open('./lab-data.json','w') as f:
    json.dump(data,f,indent=2)

In [ ]:
# Done